P1 discontinuous SWIPDG, stationary linear elliptic ESV2007 problem
==================================

This example is about approximating the solution $u$ of the elliptic problem

$$\begin{align}
  -\nabla\cdot( \kappa \nabla u ) &= f   &&\text{in } \Omega\\
                                u &= g_D &&\text{on }\partial\Omega
\end{align}$$

with datafunction as defined in `dune/gdt/test/linearelliptic/problems/ESV2007.hh` (see below) using a piecewise linear SWIPDG scheme, as in `dune/gdt/test/linearelliptic/discretizers/ipdg.hh`.

Note that the discretization below contains handling of arbitrary Dirichlet and Neumann boundary data, although the problem at hand contains only trivial Dirichlet data.

In [ ]:
from dune.xt.common import init_logger, init_mpi
init_mpi()

$$\begin{align}
  \Omega &= [-1, 1]^2\\
  \Gamma_D &= \partial\Omega\\
  \Gamma_N &= \emptyset
\end{align}$$

In [ ]:
print('initializing grid and problem ...')

from dune.xt.grid import (
    make_cube_grid__2d_simplex_aluconform as make_cube_grid,
    make_boundary_info_on_leaf_layer as make_boundary_info,
    make_apply_on_dirichlet_intersections_leaf_part as make_apply_on_dirichlet_intersections,
    make_apply_on_neumann_intersections_leaf_part as make_apply_on_neumann_intersections
)

# The estimator notebook relies on this grid cfg!
grid = make_cube_grid(lower_left=[-1, -1], upper_right=[1, 1], num_elements=[4, 4], num_refinements=2)
boundary_info = make_boundary_info(grid, 'xt.grid.boundaryinfo.alldirichlet')
apply_on_dirichlet_intersections = make_apply_on_dirichlet_intersections(boundary_info)
apply_on_neumann_intersections = make_apply_on_neumann_intersections(boundary_info)

$$\begin{align}\kappa(x) &:= 1\\
f(x) &:= \tfrac{1}{2} \pi^2 \cos(\tfrac{1}{2} \pi x_0) \cos(\tfrac{1}{2} \pi x_1)\\
g_D(x) &:= 0\end{align}$$

Note that the grid `g` is only provided to select the correct _type_ of function. These functions do not rely on the actual grid which is why we need to later on provide the grid again, i.e., for `visualize(g)`.

In [ ]:
from dune.xt.functions import (
    make_constant_function_1x1,
    make_constant_function_2x2,
    make_expression_function_1x1
)

# The estimator notebook relies on this problem cfg!
lambda_ = make_constant_function_1x1(grid, 1.0, name='lambda')
kappa = make_constant_function_2x2(grid, [[1., 0.], [0., 1.]], name='kappa')
f = make_expression_function_1x1(grid, 'x', '0.5*pi*pi*cos(0.5*pi*x[0])*cos(0.5*pi*x[1])', order=3, name='f')
g_D = make_constant_function_1x1(grid, 0.0, name='g_D')
g_N = make_constant_function_1x1(grid, 0.0, name='g_N')

In [ ]:
print('discretizing ...')

from dune.xt.la import (
    IstlRowMajorSparseMatrixDouble as Matrix,
    IstlDenseVectorDouble as Vector
)

from dune.gdt import (
    make_dg_leaf_part_to_1x1_fem_p1_space as make_dg_space,
    make_elliptic_swipdg_matrix_operator,
    make_l2_volume_vector_functional,
    make_l2_face_vector_functional,
    make_elliptic_swipdg_dirichlet_vector_functional,
    make_system_assembler,
    make_discrete_function
)

space = make_dg_space(grid)

# There are two ways to create containers: 
# * manually create them and given them to the operators/functionals
# * let those create appropriate ones

# in the CG example we chose the latter, so here we do the former
system_matrix = Matrix(space.size(), space.size(), space.compute_pattern('face_and_volume'))
swipdg_operator = make_elliptic_swipdg_matrix_operator(lambda_, kappa, boundary_info, system_matrix, space)

rhs_vector = Vector(space.size(), 0.0)
l2_force_functional = make_l2_volume_vector_functional(f, rhs_vector, space)
swipdg_dirichlet_functional = make_elliptic_swipdg_dirichlet_vector_functional(g_D, lambda_, kappa,
                                                                               boundary_info, rhs_vector,
                                                                               space)
# there are two equivalent ways to restrict the integration domain of the face functional:
# * provide an apply_on_... tag on construction
# * provide an apply_on_... tag when appending the functional to the system assembler
l2_neumann_functional = make_l2_face_vector_functional(g_N, rhs_vector, space)

# compute everything in one grid walk
system_assembler = make_system_assembler(space)
system_assembler.append(swipdg_operator)
system_assembler.append(l2_neumann_functional, apply_on_neumann_intersections)
system_assembler.append(l2_force_functional)
system_assembler.append(swipdg_dirichlet_functional)
system_assembler.assemble()

In [ ]:
print('solving ...')

from dune.xt.la import (
    IstlRowMajorSparseMatrixDouble as Matrix,
    IstlDenseVectorDouble as Vector,
    make_solver
)

# solve the linear system
u_h_vector = Vector(space.size(), 0.0)
make_solver(system_matrix).apply(rhs_vector, u_h_vector)

# visualize (this will write swipdg_esv2007_solution.vtu)
u_h = make_discrete_function(space, u_h_vector, 'u_h')
u_h.visualize('../swipdg_esv2007_solution')